In [ ]:
class Pastry:
    total_nodes = 0  # to keep track of total number of nodes
    total_nodes_add = 0  # total number of nodes added

    def __init__(self, id, x, y, t_nodes, neighbour=None):
        self.id = id
        self.x = x
        self.y = y
        self.t_nodes = t_nodes
        self.left_leaf = []  # --depicts left leaf set--#
        self.right_leaf = []  # --depicts right leaf set--#
        self.neighbour = []
        self.routing_table = [[None] * 8 for _ in range(16)]  # --create routing table
        self.data_element = []
        # --to increment the total node and total added node count
        Pastry.total_nodes += 1
        Pastry.total_nodes_add += 1

        if neighbour:
            # --nodes join the network if closest node is available--#
            self.join(neighbour, t_nodes)

    def add(self, element):
        # --to add an element to node--#
        self.data_element.append(element)

    def join(self, closest_n, t_nodes):
        # --closest_n represents closest node in the network--#
        closest_node = self.routing(closest_n, 'add', t_nodes)
        self.update_leafs(closest_node)
        self.update_neighbour(closest_n)
        self.update_table(t_nodes)

    def routing(self, closest, temp, t_nodes):
        # --simulating routing to find the closest node--#
        # --closest represents closest node to current node--#
        if temp == 'add':
            c_distance = self.calculate_distance(closest.id)
            closest_node = closest
            for node in t_nodes:
                if node.id != self.id:
                    distance = self.calculate_distance(node.id)
                    if distance < c_distance:
                        c_distance = distance
                        closest_node = node
            return closest_node

    def update_leafs(self, closest_node):
        # --to update the left and right leaf sets of node--#
        for node in closest_node.left_leaf:
            if node.id < self.id:
                self.left_leaf.append(node)
        for node in closest_node.right_leaf:
            if node.id > self.id:
                self.right_leaf.append(node)

    def update_neighbour(self, closest_n):
        if closest_n not in self.neighbour:
            self.neighbour.append(closest_n)
        for neighbours in closest_n.neighbour:
            if neighbours not in self.neighbour:
                self.neighbour.append(neighbours)

    def update_table(self, t_nodes):
        # --to update the routing table--#
        for i in range(16):
            for j in range(8):
                for node in t_nodes:
                    if node.id != self.id:
                        distance = self.calculate_distance(node.id)
                        if distance >= i and self.routing_table[i][j] is None:
                            self.routing_table[i][j] = node

    def calculate_distance(self, new_id):
        # --to calculate distance between current node and added node--#
        # --new_id is id of the other node--#
        distance = 0
        for i in range(len(self.id)):
            if self.id[i] != new_id[i]:
                break
            distance += 1
        return distance


def delete_node(node, t_nodes):
    # --to delete a node from the Pastry Network--#
    t_nodes.remove(node)
    for other_n in t_nodes:
        if node in other_n.left_leaf:
            other_n.left_leaf.remove(node)
        if node in other_n.right_leaf:
            other_n.right_leaf.remove(node)
        if node in other_n.neighbour:
            other_n.neighbour.remove(node)
        for i in range(16):
            for j in range(8):
                if other_n.routing_table[i][j] == node:
                    other_n.routing_table[i][j] = None


def routing_data(current, final_id, t_nodes, Cn, Lf):
    # -- current: node performing routing--#
    # -- final_id : ID of target node--#
    # --Cn : current hop count--#
    # --Lf: List of nodes encountered during routing--#

    distance = current.calculate_distance(final_id)
    if distance < len(final_id) and current.routing_table[distance][int(final_id[distance])] is not None:
        next_hop = current.routing_table[distance][int(final_id[distance])]
        Lf.append(next_hop)
        return routing_data(next_hop, final_id, t_nodes, Cn + 1, Lf)
    else:
        return Cn


# --to create and manage the network
if __name__ == "__main__":
    node_count = int(input("Enter No. of Nodes:"))
    lst = []

    for i in range(node_count):
        if i == 0:
            lst.append(Pastry(str(i), i, (i * (i + 1)) % 20000, lst))
        else:
            closest_node = lst[0]  # --selects the first node as the closest node--#
            lst.append(Pastry(str(i), i, (i * (i + 1)) % 20000, lst, closest_node))
    print(f"{node_count} Node are Created\n")

    while True:
        option = input("1: Exit\n2: Delete Nodes\n3: Add Elements\n4: Print Node Details\n5: Lookup Queries\n")
        if option == "1":
            break
        elif option == "2":
            deletion = int(input("How Many Nodes to Delete: "))
            if deletion <= len(lst):
                for _ in range(deletion):
                    delete_node(lst[0], lst)
        elif option == "3":
            elements = input("How many elements to Distribute: ")
            for index, i in enumerate(range(int(elements))):
                lst[index % len(lst)].add(str(index))
        elif option == "4":
            node_id = input("Enter the ID of the node: ")
            for node in lst:
                if node.id == node_id:
                    print(f"\nDetails for Node:{node.id} ")
                    print(f"Left Leaf: {[n.id for n in node.left_leaf ]}")
                    print(f"Right Leaf: {[n.id for n in node.right_leaf ]}")
                    print(f"Neighbour Nodes: {[n.id for n in node.neighbour ]}")
                    print("Routing Table:")
                    for row in node.routing_table:
                        print(row)
                    print()
                    break
        elif option == "5":
            query_node = input("Enter ID of Node to Perform Lookup\n")
            target_node = input("Enter ID of Targetted Node: ")
            for node in lst:
                if node.id == query_node:
                    lookup_hops = routing_data(node, target_node, lst, 1, [])
                    print(f"LookUp Completed in {lookup_hops} hops")
                    break
            else:
                print("Node Not Found")
        else:
            print("Choose the Right Option!!")


Enter No. of Nodes:10
10 Node are Created

1: Exit
2: Delete Nodes
3: Add Elements
4: Print Node Details
5: Lookup Queries
3
How many elements to Distribute: 66
1: Exit
2: Delete Nodes
3: Add Elements
4: Print Node Details
5: Lookup Queries
4
Enter the ID of the node: 3

Details for Node:3 
Left Leaf: []
Right Leaf: []
Neighbour Nodes: ['0']
Routing Table:
[<__main__.Pastry object at 0x7d29cf38a350>, <__main__.Pastry object at 0x7d29cf38a350>, <__main__.Pastry object at 0x7d29cf38a350>, <__main__.Pastry object at 0x7d29cf38a350>, <__main__.Pastry object at 0x7d29cf38a350>, <__main__.Pastry object at 0x7d29cf38a350>, <__main__.Pastry object at 0x7d29cf38a350>, <__main__.Pastry object at 0x7d29cf38a350>]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, 